In [60]:
import os

import numpy as np
import cv2

from sklearn.model_selection import train_test_split

from patchify import patchify

import rasterio
from rasterio.plot import reshape_as_image

import albumentations as A

PATCH_WIDTH = 128

In [61]:
raster_path = "data/T36UXV_20200406T083559_TCI_10m.jp2"
mask_path = "data/train.jp2"

with rasterio.open(raster_path, "r", driver="JP2OpenJPEG") as src:
    raster_img = src.read()
    raster_meta = src.meta

with rasterio.open(mask_path, "r", driver="JP2OpenJPEG") as src:
    mask_img = src.read()
    mask_meta = src.meta

In [62]:
raster_img = reshape_as_image(raster_img)
mask_img = reshape_as_image(mask_img)

In [63]:
IM_WIDTH = IM_HEIGHT = 10752
raster_img = cv2.resize(raster_img, (IM_HEIGHT, IM_WIDTH))
mask_img = cv2.resize(mask_img, (IM_HEIGHT, IM_WIDTH))
mask_img = np.expand_dims(mask_img, axis=2)

raster_patches = patchify(raster_img, (PATCH_WIDTH, PATCH_WIDTH, 3), PATCH_WIDTH)
mask_patches = patchify(mask_img, (PATCH_WIDTH, PATCH_WIDTH, 1), PATCH_WIDTH)

In [64]:
len(raster_patches), type(raster_patches), raster_patches.shape

(84, numpy.ndarray, (84, 84, 1, 128, 128, 3))

In [65]:
42*0.35

14.7

In [66]:
raster_patches_train, raster_patches_test, mask_patches_train, mask_patches_test = \
    train_test_split(raster_patches, mask_patches, test_size=0.3, random_state=42)

In [67]:
len(raster_patches_test), type(raster_patches_test), raster_patches_test.shape

(26, numpy.ndarray, (26, 84, 1, 128, 128, 3))

In [68]:
len(raster_patches_train), type(raster_patches_train), raster_patches_train.shape

(58, numpy.ndarray, (58, 84, 1, 128, 128, 3))

In [69]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.GridDistortion(p=0.5)
])

In [70]:
m, n, *_ = raster_patches_train.shape
train_path = 'data/patches/train/augmentation_0'
os.makedirs(train_path, exist_ok=True)
os.makedirs(os.path.join(train_path, "images"), exist_ok=True)
os.makedirs(os.path.join(train_path, "masks"), exist_ok=True)

for i in range(m):
    for j in range(n):
        raster_patch = raster_patches_train[i, j, 0]
        mask_patch = mask_patches_train[i, j, 0]

        cv2.imwrite(os.path.join(train_path, "images", f"patch_{i}_{j}.jpg"), cv2.cvtColor(raster_patch, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(train_path, "masks", f"mask_{i}_{j}.jpg"), cv2.cvtColor(mask_patch, cv2.COLOR_RGB2BGR))

In [ ]:
m, n, *_ = raster_patches_test.shape
test_path = 'data/patches/test'
os.makedirs(test_path, exist_ok=True)
os.makedirs(os.path.join(test_path, "images"), exist_ok=True)
os.makedirs(os.path.join(test_path, "masks"), exist_ok=True)

for i in range(m):
    for j in range(n):
        raster_patch = raster_patches_test[i, j, 0]
        mask_patch = mask_patches_test[i, j, 0]

        cv2.imwrite(os.path.join(test_path, "images", f"patch_{i}_{j}.jpg"), cv2.cvtColor(raster_patch, cv2.COLOR_RGB2BGR))
        cv2.imwrite(os.path.join(test_path, "masks", f"mask_{i}_{j}.jpg"), cv2.cvtColor(mask_patch, cv2.COLOR_RGB2BGR))

In [ ]:
augmentations = 4
for a in range(1, augmentations+1):

    augmentation_path = f"data/patches/train/augmentation_{a}"
    os.makedirs(augmentation_path, exist_ok=True)
    os.makedirs(os.path.join(augmentation_path, "images"), exist_ok=True)
    os.makedirs(os.path.join(augmentation_path, "masks"), exist_ok=True)

    for i in range(m):
        for j in range(n):
            raster_patch = raster_patches_train[i, j, 0]
            mask_patch = mask_patches_train[i, j, 0]
            transformed = transform(image=raster_patch, mask=mask_patch)

            fname = f"patch_raster_{i}_{j}.png"
            cv2.imwrite(os.path.join(augmentation_path, "images", fname), cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR))

            fname = f"patch_mask_{i}_{j}.png"
            cv2.imwrite(os.path.join(augmentation_path, "masks", fname), cv2.cvtColor(transformed['mask'], cv2.COLOR_RGB2BGR))
